In [ ]:
import xml.etree.ElementTree as ET
import xml.dom.minidom

# Function to create a new event
def create_event(time, event_type, **kwargs):
    event = ET.Element("event", time=str(time), type=event_type)
    for key, value in kwargs.items():
        if value is not None:  # Ensure value is not None
            event.set(key, value)
    return event

# Function to extract unique person IDs and coordinates
def extract_unique_person_data(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    person_data = {}

    for event in root.findall("event"):
        person = event.get("person")
        if person and person not in person_data:
            x = event.get("x")
            y = event.get("y")
            person_data[person] = {"x": x, "y": y}
    
    return person_data

# Function to generate new events for each person
def generate_events_for_person(person_data, base_times, time_intervals):
    new_events = []
    for person, coords in person_data.items():
        x = coords["x"]
        y = coords["y"]
        link = "1"  # Assuming link ID starts from 1
        vehicle = person  # Assuming vehicle ID is the same as person ID

        for base_time in base_times:
            new_events.append(create_event(base_time + time_intervals[0], "actend", person=person, link=link, x=x, y=y, actType="work"))
            new_events.append(create_event(base_time + time_intervals[0], "departure", person=person, link=link, legMode="car", computationalRoutingMode="car"))
            new_events.append(create_event(base_time + time_intervals[0], "PersonEntersVehicle", person=person, vehicle=vehicle))
            new_events.append(create_event(base_time + time_intervals[0], "vehicle enters traffic", person=person, link=link, vehicle=vehicle, networkMode="car",
                                            relativePosition="1.0"))
            new_events.append(create_event(base_time + time_intervals[1], "left link", link=link, vehicle=vehicle))
            new_events.append(create_event(base_time + time_intervals[1], "entered link", link=str(int(link) + 1), vehicle=vehicle))
            new_events.append(create_event(base_time + time_intervals[2], "left link", link=str(int(link) + 1), vehicle=vehicle))
            new_events.append(create_event(base_time + time_intervals[2], "entered link", link=str(int(link) + 2), vehicle=vehicle))
            new_events.append(create_event(base_time + time_intervals[3], "vehicle leaves traffic", person=person, link=str(int(link) + 2), vehicle=vehicle,
                                            networkMode="car", relativePosition="1.0"))
            new_events.append(create_event(base_time + time_intervals[3], "PersonLeavesVehicle", person=person, vehicle=vehicle))
            new_events.append(create_event(base_time + time_intervals[3], "arrival", person=person, link=str(int(link) + 2), legMode="car", vehicle=vehicle))
            new_events.append(create_event(base_time + time_intervals[3], "actstart", person=person, link=str(int(link) + 2), x=x, y=y, actType="work"))

    return new_events

# Main function to generate the modified XML
def create_modified_xml(input_file, output_file):
    person_data = extract_unique_person_data(input_file)
    base_times = [28800.0, 63000.0]
    time_intervals = [0.0, 1.0, 20.0, 40.0]

    # Create new root element
    root = ET.Element("events", version="1.0")

    # Generate new events for each unique person
    new_events = generate_events_for_person(person_data, base_times, time_intervals)
    for event in new_events:
        root.append(event)

    # Convert the ElementTree to a string
    xml_str = ET.tostring(root, encoding='utf-8', method='xml')

    # Use minidom to prettify the XML string
    dom = xml.dom.minidom.parseString(xml_str)
    pretty_xml_str = dom.toprettyxml(indent="    ")

    # Write the prettified XML to the file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(pretty_xml_str)

    print("XML modification complete.")

# Run the function
create_modified_xml('output_events_building.xml', 'bulding_modified.xml')
